In [ ]:
!hf auth login

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: write).
The token `learning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `learning`


In [ ]:
# Load model directly
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
device = "cpu"

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it")

# tokenizer = tokenizer.to(device)
model = model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [20]:
batch = [
    # [{"role": "user", "content": "What are you?"}],
    [{"role": "user", "content": "Explain attention masks"}],
    [{"role": "user", "content": "Tell me a story"}],
    [{"role": "user", "content": "How does a car engine work?"}],
]

encoded = tokenizer.apply_chat_template(
    batch,
    add_generation_prompt=True,
    tokenize=True,
    padding=True,            # THIS MAKES IT A BATCH
    return_dict=True,
    return_tensors="pt",
)
encoded = {k: v.to(device) for k, v in encoded.items()}

In [21]:
with torch.no_grad():
    outputs = model(
        input_ids=encoded["input_ids"].to(device),
        attention_mask=encoded["attention_mask"].to(device),
        return_dict=True,
        output_hidden_states=True,
    )

In [22]:
outputs.logits.shape

torch.Size([3, 16, 262144])

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

class ProbingDatasetGenerator:
    """A class to generate model outputs for probing datasets."""
    def __init__(self, model_name, device="cuda"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)

        self.model = self.model.to(device)
        self.device = device
    
    def _generate_batch(self, batch):
        """
        Generates model outputs for a batch of inputs.
        
        Args:
            batch (list): A list of input data formatted for the tokenizer.

        Returns:
            outputs: The model outputs after processing the batch.
        """
        encoded = self.tokenizer.apply_chat_template(
            batch,
            add_generation_prompt=True,
            tokenize=True,
            padding=True,
            return_dict=True,
            return_tensors="pt",
        )
        encoded = {k: v.to(self.device) for k, v in encoded.items()}

        with torch.no_grad():
            outputs = self.model(
                input_ids=encoded["input_ids"].to(self.device),
                attention_mask=encoded["attention_mask"].to(self.device),
                return_dict=True,
                output_hidden_states=True,
            )
        return outputs

    def get_activations(self, batch):
        """
        Retrieves hidden states.
        
        Args:
            batch (list): A list of input data formatted for the tokenizer.

        Returns:
            hidden_states: The hidden states.
        """

        outputs = self._generate_batch(batch)
        return outputs.hidden_states

    def store_activations(self, hidden_states, label, filepath):
        """
        Stores hidden states from a specific layer to a file.
        
        Args:
            layer_idx (int): The index of the layer from which to extract hidden states.
            batch (list): A list of input data formatted for the tokenizer.
            filepath (str): The path to the file where hidden states will be stored.
        """
        data = {
            "activations": {
                f"hidden_state{i}": hidden_state.cpu() for i, hidden_state in enumerate(hidden_states)
            },
            "label": label,
        }

        torch.save(data, filepath)

In [3]:
generator = ProbingDatasetGenerator(
    model_name="google/gemma-3-1b-it",
    device="cpu"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [4]:
import torch
batch = [
    # [{"role": "user", "content": "What are you?"}],
    [{"role": "user", "content": "Explain attention masks"}],
    [{"role": "user", "content": "Tell me a story"}],
    [{"role": "user", "content": "How does a car engine work?"}],
]

activations = generator.get_activations(batch=batch)

In [7]:
generator.store_activations(activations, 1, "layers_activations.pt")

In [8]:
tensors = torch.load("layers_activations.pt")

In [25]:
import kagglehub

class ProbingDatasetLoader:
    """Class to download and load the probing dataset from Kaggle."""
    def download_dataset(self, dataset_name) -> str:
        """
        Downloads and loads the dataset from the specified filepath.
        
        Returns:
            path: Path to the downloaded dataset.
        """
        path = kagglehub.dataset_download(dataset_name)
        self.dataset_folder = path
        return path

    def load_dataset(self) -> list:
        """
        Loads the dataset from the specified filepath.
        
        Returns:
            data: The loaded dataset as a list of examples.
        """
        path = self.dataset_folder + "/data_set_4.csv"
        with open(path, 'r') as file:
            data = file.readlines()
        return data

In [26]:
dataset_loader = ProbingDatasetLoader()
# dataset_loader.download_dataset("zepopo/ukrainian-fake-and-true-news")

In [28]:
dataset_loader.dataset_folder = "/kaggle/input/ukrainian-fake-and-true-news"

In [29]:
dataset = dataset_loader.load_dataset()
dataset

[',Link,Text,Label\n',
 '0,https://t.me/c/1376264484/328,"Просто слухайте цей діалог. Ні, це не нарізка і фільм Тарантіно.",True\n',
 '1,https://t.me/c/1376264484/338,️ Рубль став найнестабільнішою валютою у всьому світі,True\n',
 '2,https://t.me/c/1376264484/350,Перше звернення мера Мелітополя Івана Федорова після звільнення,True\n',
 '3,https://t.me/c/1376264484/361,"Росія загрожує Боснії ""українським сценарієм"" через її можливе вступ до НАТО.",True\n',
 '4,https://t.me/c/1376264484/366,"Енергоатом повідомив, що окупанти пошкодили високовольтну лінію електропередач Запорізька АЕС — Каховська.",True\n',
 '5,https://t.me/c/1376264484/376,Макрон закликає готуватися до великої війни у \u200b\u200bЄвропі:,True\n',
 '6,https://t.me/c/1376264484/386,"Байден має намір заявити Сі Цзіньпіну, що США вживуть заходів, якщо з\'ясується, що Китай підтримує дії Росії в Україні",True\n',
 '7,https://t.me/c/1376264484/391,Урядові літаки РФ здійснили незвичайні рейси з Москви до Сибіру,True\n',
 '8,h

In [19]:
! cd /root/.cache/kagglehub/datasets/zepopo/ukrainian-fake-and-true-news/versions/2
! ls /root/.cache/kagglehub/datasets/zepopo/ukrainian-fake-and-true-news/versions/2

data_set_4.csv	news_data.csv


In [ ]:
! 

layer5_activations.pt  layers_activations.pt  sample_data
